In [25]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)
smtp_configuration = {
    "server": os.getenv("SMTP_SERVER"),
    "port": os.getenv("SMTP_PORT"),
    "username": os.getenv("SMTP_USERNAME"),
    "password": os.getenv("SMTP_PASSWORD"),
    "from": os.getenv("SMTP_SENDER_EMAIL")
}

Lets Set up the tools

In [26]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.utils import formatdate
import smtplib
from langchain_community.tools import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper

# Create the Serper search tool
search = GoogleSerperAPIWrapper()
serper_tool = Tool(
    name="Search",
    func=search.run,
    description="Useful for when you need more information from an online search"
)

def push(text: str):
     """Send a push notification as an email to the user"""
     msg = MIMEMultipart()
     msg['From'] = smtp_configuration["from"]
     msg['To'] = "nobelnurulislam@gmail.com"
     msg['Subject'] = "New Notification from LangGraph"
     msg['Date'] = formatdate(localtime=True) 

     msg.attach(MIMEText(text, 'html'))

     with smtplib.SMTP_SSL(smtp_configuration["server"], smtp_configuration["port"]) as server:
            server.login(smtp_configuration["username"], smtp_configuration["password"])
            # Combine all recipients
            recipients = ["nobelnurulislam@gmail.com"]
            server.sendmail(smtp_configuration["from"], recipients, msg.as_string())

tool_push = Tool(
    name="send_push_notification",
    func=push,
    description="Sends a push notification to the user via email"
)

tools = [serper_tool, tool_push]

Initialize the Graph

In [27]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

Lets create the nodes

In [28]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import ToolNode

def chatbot(state: State):
    llm = ChatOpenAI(model_name="gpt-4o-mini")
    llm_with_tools = llm.bind_tools(tools)
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

toolNode = ToolNode(tools=tools)

Build the graph

In [29]:
from langgraph.graph import START, END
from langgraph.prebuilt import tools_condition

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", toolNode)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge("chatbot", END)

Adding Sqlite memory and compile the graph

In [ ]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "memory.db"
conn = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(conn)

graph = graph_builder.compile(checkpointer=sql_memory)
display(Image(graph.get_graph().draw_mermaid_png()))

Time to Use the gradio to run the chat interface

In [ ]:
import gradio as gr

config = {"configurable": {"thread_id": "1"}}

def chat(message, history):
    response = graph.invoke({"messages": [{"role": "user", "content": message}]}, config)
    return response["messages"][-1].content


chat_interface = gr.ChatInterface(chat, type="messages", theme="soft")
chat_interface.launch()